In [ ]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import multiscale.ultrasound.reconstruction as recon

import multiscale.itk.itk_plotting as iplt
import multiscale.itk.registration as reg
import multiscale.itk.process as proc
import multiscale.utility_functions as util

import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

In [ ]:
def open_us(us_path, dynamic_range, spacing, origin):
    """Open the US image, window it to a dynamic range, and rotate it to microscope coordinate axes"""
    raw_image = sitk.ReadImage(str(us_path))
    windowed_image = proc.window_image(raw_image, dynamic_range)
    us_image = rotate_axes_to_microscope(windowed_image)
    us_image.SetSpacing(spacing)
    us_image.SetOrigin(origin)
    us_image.SetDirection([1, 0, 0, 0, 1, 0, 0, 0, -1])
    return us_image

def open_mpm(mpm_path, mpm_origin_path, mpm_spacing):
    """Open the MPM image and set the direction to -1 in Z to mirror microscope convention"""
    positions = positions_from_ometif(mpm_origin_path)
    origin = np.min(positions, 0)
    mpm_image = sitk.ReadImage(str(mpm_path))
    mpm_image.SetSpacing(mpm_spacing)
    mpm_image.SetOrigin(origin)
    mpm_image.SetDirection([1, 0, 0, 0, 1, 0, 0, 0, -1])
    return mpm_image

def rotate_axes_to_microscope(image):
    """Rotate the US axes to be along the microscope axes"""
    arr = sitk.GetArrayFromImage(image)
    arr_rot = np.swapaxes(arr, 0, 1)
    arr_rot = np.flip(arr_rot, 0).astype(np.uint8)
    return sitk.GetImageFromArray(arr_rot)

def positions_from_ometif(file_path):
    """Read a .ome.tif file and grab the image positions as a numpy array"""
    reader = sitk.ImageFileReader()
    reader.SetFileName(str(file_path))
    reader.ReadImageInformation()
    raw_info = reader.GetMetaData('ImageDescription')
    info = tif.xml2dict(raw_info)
    mpm_list = []
    for position in info['OME']['Image']:
        x = position['StageLabel']['X']
        y = position['StageLabel']['Y']
        z = -1*position['Pixels']['Plane'][0]['PositionZ']
        mpm_list.append(np.array([x, y, z]))
    return np.array(mpm_list)

def get_xy_origin(pl_path):
    """Read an ultrasound position list and get the XY origin"""
    raw_pos_list = util.read_json(us_pl_path)
    pos_list = recon.clean_position_text(raw_pos_list)[0]
    xy_origin = np.min(pos_list, 0)
    return xy_origin

# Rotation

In [ ]:
# All US images are on the same positions list for X and Y.  Each has a slightly different Z height based on the indicator gauge
us_pl_path = Path(r'C:\Users\mpinkert\Box\Research\LINK\Phantom Trials\2019-05-04\2019-05-04_US - 3X 100YSep.pos')
us_xy_origin = get_xy_origin(us_pl_path)

# These Z heights are recorded from the gauge and are in microns.
us_heights = [0, -6, -6, -7, -6]
us_origins = [np.array([us_xy_origin[0], us_xy_origin[1], us_heights[idx]]) for idx in range(5)]

In [ ]:
# The MPM positions 
metadata_dir = Path(r'F:\Research\LINK\Phantom Trials\Fiducial paper analysis\Metadata images')
mpm_origins = [Path(metadata_dir, 'Fiducial acq ' + str(idx+1) + '.ome.tif') for idx in range(5)]

In [ ]:
us_spacing = [25, 25, 25]

#In this case, we are using MPM images downsampled by 8x in XY
mpm_spacing = [8.16, 8.16, 25]

dynamic_range = 50

In [ ]:
# Specify where the images are.
mpm_dir = Path(r'F:\Research\LINK\Phantom Trials\Fiducial paper analysis\MPM Images\MPM downsampled')
mpm_paths = [Path(mpm_dir, 'MPM acq {}_8x.tif'.format(str(idx))) for idx in range(1, 6)]

us_dir = Path(r'F:\Research\LINK\Phantom Trials\Fiducial paper analysis\Ultrasound')
us_paths = [Path(us_dir, 'US Rotation {}.tif'.format(str(idx))) for idx in range(1, 6)]

In [ ]:
mpm_images = [open_mpm(mpm_paths[idx], mpm_origins[idx], mpm_spacing) for idx in range(5)]

In [ ]:
us_images = [open_us(us_paths[idx], dynamic_range, us_spacing, us_origins[idx]) for idx in range(5)]

In [ ]:
# Acquire points manually for the XY registrations.
points = []

In [ ]:
points.append(iplt.RegistrationPointDataAcquisition(mpm_images[0], us_images[0]))

In [ ]:
points.append(iplt.RegistrationPointDataAcquisition(mpm_images[1], us_images[1]))

In [ ]:
points.append(iplt.RegistrationPointDataAcquisition(mpm_images[2], us_images[2]))

In [ ]:
points.append(iplt.RegistrationPointDataAcquisition(mpm_images[3], us_images[3]))

In [ ]:
points.append(iplt.RegistrationPointDataAcquisition(mpm_images[4], us_images[4]))

In [ ]:
fixed_points = []
moving_points = []
for idx in range(5):
    fixed, moving = points[idx].get_points_flat()
    fixed_points.append(fixed)
    moving_points.append(moving)

In [ ]:
fiducial_dir = Path(r'F:\Research\LINK\Phantom Trials\Fiducial paper analysis')
fixed_points_path = Path(fiducial_dir, 'Rotation init points - fixed.txt')
moving_points_path = Path(fiducial_dir, 'Rotation init points - moving.txt')

In [ ]:
len(fixed_points[0])

In [ ]:
# Points previously acquired
if len(fixed_points[0]) == 0:
    with open(fixed_points_path, 'rb') as fp:
        fixed_points = pickle.load(fp)   
    with open(moving_points_path, 'rb') as fp:
        moving_points = pickle.load(fp)
else:
    with open(fixed_points_path, 'wb') as fp:
        pickle.dump(fixed_points, fp)
    with open(moving_points_path, 'wb') as fp:
        pickle.dump(moving_points, fp)

In [ ]:
final_params = []
metrics = []
stops = []

In [ ]:
for idx in range(5):
    initial_transform = sitk.LandmarkBasedTransformInitializer(sitk.VersorRigid3DTransform(), fixed_points[idx], moving_points[idx])
    final_transform, metric, stop = reg.register(mpm_images[idx], us_images[idx], initial_transform=initial_transform)
    final_params.append(final_transform.GetParameters())
    metrics.append(metric)
    stops.append(stop)
    print('Finished registration {}'.format(idx+1))

In [ ]:
rotation_params_path = Path(fiducial_dir, 'Rotation final parameters.txt')

if len(final_params) == 0:
    with open(rotation_params_path, 'rb') as fp:
        final_params = pickle.load(fp)
else:
    with open(rotation_params_path, 'wb') as fp:
        pickle.dump(final_params, fp)

In [ ]:
transform_mean = np.mean(np.array(final_params), 0)
transform_std = np.std(np.array(final_params), 0)
print(transform_mean)
print(transform_std)

In [ ]:
registered_images = []
for idx in range(5):
    transform = sitk.VersorRigid3DTransform()
    transform.SetParameters(final_params[idx])
    registered_images.append(sitk.Resample(us_images[idx], mpm_images[idx], transform, sitk.sitkLinear, 0.0, mpm_images[idx].GetPixelID()))

In [ ]:
registered_dir = Path(r'F:\Research\LINK\Phantom Trials\Fiducial paper analysis\Ultrasound\Registered')


In [ ]:
for idx in range(5):
    reg_path = Path(registered_dir, 'US Rot Reg {}.tif'.format(idx+1))
    sitk.WriteImage(registered_images[idx], str(reg_path))

In [ ]:
fig, ax = plt.subplots(3, 2)
for idx in range(5):
    if idx == 0:
        slice = 4
    else:
        slice = 1
    overlay = iplt.overlay_images(registered_images[idx], mpm_images[idx],  slice=slice)
    ax[int(idx % 3), int(np.floor(idx/3))].imshow(overlay)
    ax[int(idx % 3), int(np.floor(idx/3))].axis('off')
ax[2, 1].axis('off')

# Height determination

In [ ]:
def connected_components(us_image):
    """Process the US image using Otsu thresholding and binary opening/closing to get the connected components"""
    thresh_filter = sitk.OtsuThresholdImageFilter()
    thresh_filter.SetInsideValue(0)
    thresh_filter.SetOutsideValue(1)
    thresh_img = thresh_filter.Execute(us_image)
    thresh_value = thresh_filter.GetThreshold()

    print("Threshold used: " + str(thresh_value))

    cleaned_thresh_img = sitk.BinaryOpeningByReconstruction(thresh_img, [4, 4, 2])
    cleaned_thresh_img = sitk.BinaryClosingByReconstruction(cleaned_thresh_img, [4, 4, 2])

    connected_img = sitk.ConnectedComponent(cleaned_thresh_img)
    return connected_img

def get_fiducial_stats(connected_img):
    """Get statistics for each object in a label image"""
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.ComputeOrientedBoundingBoxOn()
    stats.ComputePerimeterOn()
    stats.Execute(connected_img)    
    return stats

def filter_labels(stats):
    """Filter labels by property so you only get the fiducial circles"""
    return [l for l in stats.GetLabels() if (stats.GetNumberOfPixels(l) < 200000 
                                             and stats.GetEquivalentEllipsoidDiameter(l)[1] > 2000)]

def get_leveled_centroid(stats, true_labels):
    """Get the leveled Z height, equivalent to the bottom of fiducial, for each label"""
    centroid = [stats.GetCentroid(l) for l in true_labels]
    level_center = []
    idx = 0
    for center in centroid:
        level_center.append(center[2] + np.floor((idx+3)/3)*1000)
        idx = idx+1
    return np.array(level_center)

def get_ellipsoid_radius(stats, true_labels):
    rad = [0.5*stats.GetEquivalentEllipsoidDiameter(l)[0] for l in true_labels]
    return rad

def get_leveled_top(stats, true_labels):
    """Get the indexes of the top of the bounding box, leveled to be equivalent to the bottom of the fiducial"""
    boxes = [stats.GetBoundingBox(l) for l in true_labels]
    top = []
    for box in boxes:
        top.append(box[2] + box[5])
    level_top = []
    for idx in range(len(top)):
        level_top.append(top[idx] - np.floor((idx+3)/3)*40)

    return np.array(level_top)

def get_leveled_bottom(stats, true_labels):
    """Get the indexes of the bottom of the bounding box, leveled to be equivalent to the bottom of the fiducial"""
    boxes = [stats.GetBoundingBox(l) for l in true_labels]
    bottom = []
    for box in boxes:
        bottom.append(box[2])
    level_bottom = []
    for idx in range(len(bottom)):
        level_bottom.append(bottom[idx] - np.floor((idx+3)/3)*40)

    return np.array(level_bottom)

def apply_transform(us_image, mpm_image, transform_params):
    transform = sitk.VersorRigid3DTransform()
    transform.SetParameters(transform_params)
    reg = sitk.Resample(us_image, mpm_image, transform, sitk.sitkLinear, 0.0, mpm_image.GetPixelID())
    return reg

In [ ]:
rot_conn = None

In [ ]:
rot_conn = [connected_components(image[:, :, 40:165]) for image in us_images]

In [ ]:
# Open/Save the images if we are skipping re-calculating them
if rot_conn is None:
    rot_conn = []
    read = True
else:
    read=False
    
segment_dir = Path(r'F:\Research\LINK\Phantom Trials\Fiducial paper analysis\Segmented')

for idx in range(len(rot_conn)):
    conn_path = Path(segment_dir, 'Rot Segment {}.tif'.format(idx+1))
    if read:
        rot_conn.append(sitk.Cast(sitk.ReadImage(str(conn_path)), sitk.sitkUInt32))
    else:
        sitk.WriteImage(sitk.Cast(rot_conn[idx], sitk.sitkUInt8), str(conn_path))

In [ ]:
rot_stats = [get_fiducial_stats(image) for image in rot_conn]
rot_labels = [filter_labels(stat) for stat in rot_stats]
rot_centroids = np.array([get_leveled_centroid(rot_stats[idx], rot_labels[idx]) for idx in range(len(rot_stats))])
rot_rad = np.array([get_ellipsoid_radius(rot_stats[idx], rot_labels[idx]) for idx in range(len(rot_stats))])

In [ ]:
fiducial_bottom = np.mean(rot_centroids) + np.mean(rot_rad)
bottom_std = np.std(rot_centroids)
print(fiducial_bottom)
print(bottom_std)

In [ ]:
# One note to be cautious on: Right now, increasing negative is further into the sample in stage terms,
# but is below sample in ITK terms

In [ ]:
height_transforms = [transform_mean.copy() for idx in range(4)]

In [ ]:
for idx in range(4):
    height_transforms[idx][5] = -1*(mpm_images[0].GetOrigin()[2] - fiducial_bottom + 1000*idx)

In [ ]:
height_reg = [apply_transform(us_images[4], mpm_images[4], height_transforms[idx]) for idx in range(4)]

In [ ]:
mpm_reg = [apply_transform(mpm_images[4], us_images[4], -1*height_transforms[idx]) for idx in range(4)]

In [ ]:
for idx in range(4):
    reg_path = Path(registered_dir, 'MPM Height Reg {}mm.tif'.format(idx))
    sitk.WriteImage(mpm_reg[idx], str(reg_path))

In [ ]:
mpm_all_reg = mpm_reg[0] + mpm_reg[1] + mpm_reg[2] + mpm_reg[3]


In [ ]:
overlay_array2 = iplt.overlay_images(mpm_all_reg, us_images[4])
overlay_img2 = sitk.GetImageFromArray(overlay_array)
overlay_img2.CopyInformation(us_images[4])
iplt.MultiImageDisplay([overlay_img2])

In [ ]:
reg_path = Path(registered_dir, 'MPM all reg.tif')
sitk.WriteImage(mpm_all_reg, str(reg_path))

In [ ]:
center_transforms = [transform_mean.copy() for idx in range(4)]
for idx in range(4):
    center_transforms[idx][5] = -1*(mpm_images[0].GetOrigin()[2] - fiducial_bottom - np.mean(rot_rad) + 1000*idx)

In [ ]:
center_reg = [apply_transform(us_images[4], mpm_images[4], center_transforms[idx]) for idx in range(4)]

In [ ]:
center_img = center_reg[0] + center_reg[1] + center_reg[2] + height_reg[3]
reg_path = Path(registered_dir, 'US Fiducial center on MPM grid.tif')
sitk.WriteImage(center_img, str(reg_path))

In [ ]:
array = sitk.GetArrayFromImage(center_img)

In [ ]:
center_overlay = iplt.overlay_images(mpm_images[4], center_img, slice=1)
plt.imshow(center_overlay)

## Tendon

In [ ]:
tendon_us_path = Path(us_dir, 'Tendon.tif')
tendon_origin = [us_xy_origin[0], us_xy_origin[1], 4]
us_tendon = open_us(tendon_us_path, dynamic_range, us_spacing, tendon_origin)

tendon_mpm_path = Path(mpm_dir, 'Tendon_8x.tif')
tendon_origin_path = Path(metadata_dir, 'Tendon.ome.tif')
mpm_tendon = open_mpm(tendon_mpm_path, tendon_origin_path, mpm_spacing)

In [ ]:
tendon_reg = apply_transform(us_tendon, mpm_tendon, height_transforms[0])
#reg_path = Path(registered_dir, 'US Tendon.tif')
#sitk.WriteImage(tendon_reg, str(reg_path))

In [ ]:
tendon_mpm_reg = apply_transform(mpm_tendon, us_tendon, -1*height_transforms[0])

In [ ]:
overlay_array = iplt.overlay_images(tendon_mpm_reg, us_tendon)
overlay_img = sitk.GetImageFromArray(overlay_array)
overlay_img.CopyInformation(us_tendon)
iplt.MultiImageDisplay([overlay_img])

# Off focus determination

In [ ]:
height_image_names = [image for image in os.listdir(us_dir) if (image.startswith('US_H'))]
height_image_names.append('US Rotation 1.tif')
height_z = [-4, -4, -10, -10, -2, -2, 12, 12, 7, 7, 2, 2, 0]
height_images = [open_us(Path(us_dir, height_image_names[idx]), dynamic_range, us_spacing, 
                         np.array([us_xy_origin[0], us_xy_origin[1], height_z[idx]]))
                 for idx in range(len(height_z))]

In [ ]:
height_images[0] = open_us(Path(us_dir, height_image_names[0]), dynamic_range, us_spacing, 
                         np.array([us_xy_origin[0], us_xy_origin[1], height_z[0]]))

In [ ]:
off_focus = [name.endswith('7mm.tif') for name in height_image_names]

In [ ]:
off_focus

In [ ]:
height_image_names

In [ ]:
connected_images = [connected_components(image) for image in height_images]

In [ ]:
stats = [get_fiducial_stats(image) for image in connected_images]
true_labels = [filter_labels(stat) for stat in stats]
centroids = [get_leveled_centroid(stats[idx], true_labels[idx]) for idx in range(len(stats))]
tops = [get_leveled_top(stats[idx], true_labels[idx]) for idx in range(len(stats))]

In [ ]:
offfoc_centroids = []
onfoc_centroids = []
for idx in range(len(centroids)):
    if off_focus[idx]:
        if len(centroids[idx]) == 9:
            offfoc_centroids.append(centroids[idx])
    else:
        onfoc_centroids.append(centroids[idx])

In [ ]:
np.std(np.array(onfoc_centroids)[:, 0:5])

In [ ]:
np.std(np.array(offfoc_centroids)[:, 0:5])

In [ ]:
for center in centroids:
    print(center)

In [ ]:
valid_centroids = []
for center in centroids:
    if len(center) == 9:
        valid_centroids.append(center)
    
valid_centroids = np.array(valid_centroids)

In [ ]:
np.std(valid_centroids[:, 0:5])

In [ ]:
valid_tops = []
for top in tops:
    if len(top) == 9:
        valid_tops.append(top)
        
valid_tops = np.array(valid_tops)

In [ ]:
np.std(valid_tops[:, 0:5])

In [ ]:
bottoms = [get_leveled_bottom(stats[idx], true_labels[idx]) for idx in range(len(stats))]

In [ ]:
valid_bottoms = []
for bottom in bottoms:
    if len(bottom) == 9:
        valid_bottoms.append(top)
        
valid_bottoms = np.array(valid_tops)

In [ ]:
print(np.std(valid_bottoms))
print(np.std(valid_bottoms[:, 0:5]))

In [ ]:
iplt.MultiImageDisplay(image_list=[sitk.LabelOverlay(height_images[9], connected_images[9])], title_list=[height_image_names[9]])

In [ ]:
iplt.MultiImageDisplay(image_list=[sitk.LabelOverlay(height_images[8], connected_images[8])], title_list=[height_image_names[8]])

In [ ]:
off_focus = [name for name in height_image_names if name.endswith('7mm.tif')]
on_focus = [name for name in height_image_names if not name.endswith('7mm.tif')]

In [ ]:
height_images[9].GetSize()

In [ ]:
full_size = height_images[9].GetSize()
roi_size = [full_size[0], full_size[1], full_size[2] - 80]
index = [0, 0, 40]

In [ ]:
cutdown_image = height_images[9][:, :, 40:180]

In [ ]:
conn = connected_components(cutdown_image)

In [ ]:
iplt.MultiImageDisplay(image_list=[sitk.LabelOverlay(cutdown_image, conn)], title_list=[height_image_names[9]])

In [ ]:
st = get_fiducial_stats(conn)

In [ ]:
tl = filter_labels(st)

In [ ]:
tl

In [ ]:
cet = get_leveled_centroid(st, tl)

In [ ]:
cet

In [ ]:
conn.GetOrigin()

In [ ]:
iplt.MultiImageDisplay(image_list=[sitk.LabelOverlay(height_images[9], conn)], title_list=[height_image_names[9]])

In [ ]:
cutdown_image.GetOrigin()

In [ ]:
height_images[9].GetOrigin()

In [ ]:
np.std(np.array(rot_centroids)[:, 0:5])

In [ ]:
help(sitk.ChangeLabelImageFilter)

In [ ]:
conn